# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Defaulting to user installation because normal site-packages is not writeable


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [10]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位经验丰富的生物学教授。
你对生物学的各个分支都有深入的了解，特别擅长解释复杂的生物学概念。
当遇到未知的生物学问题时，你会利用你的专业知识来提供最合理的解释。

这是一个问题：
{input}"""

computer_science_template = """你是一位杰出的计算机科学教授。
你对算法、数据结构、编程语言和最新的技术趋势都有深刻的理解。
面对计算机科学的问题，你总能提供清晰、准确的答案。

这是一个问题：
{input}"""

chinese_literature_template = """你是一位对汉语言文学充满热情的教授。
你对古典文学和现代文学都有着深厚的研究，能够洞察文学作品的深层意义。
在解读文学作品或讨论语言学问题时，你总能提供独到的见解。

这是一个问题：
{input}"""


In [11]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物学问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机科学问题",
        "prompt_template": computer_science_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于讨论汉语言文学相关的问题",
        "prompt_template": chinese_literature_template,
    },
]

In [12]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", base_url="https://api.xiaoai.plus/v1")

In [13]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [14]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [15]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [16]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [17]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物学问题', '计算机: 适用于回答计算机科学问题', '汉语言文学: 适用于讨论汉语言文学相关的问题']


In [18]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机: 适用于回答计算机科学问题
汉语言文学: 适用于讨论汉语言文学相关的问题


In [19]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [20]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机: 适用于回答计算机科学问题
汉语言文学: 适用于讨论汉语言文学相关的问题

<< INPUT >>
{inp

In [21]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [22]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '什么是黑体辐射？?'}
> Finished chain.
{'input': '什么是黑体辐射？?', 'text': '\n黑体辐射是指一个完美吸收和辐射的理想物体所产生的电磁辐射。根据普朗克定律，黑体辐射的能量密度与波长的关系可以用黑体辐射公式来描述，它是热力学中重要的概念。黑体辐射的能谱是连续的，而且具有特定的形状，称为黑体辐射谱，它的峰值波长与温度有关。黑体辐射在宏观和微观的物理现象中都有重要应用，例如太阳辐射、宇宙背景辐射和热辐射。'}


In [16]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们需要确定大于40的质数。40不是质数，我们从41开始检查。41是质数，但41+1=42不能被3整除。接下来是43，43+1=44也不能被3整除。然后是47，47+1=48可以被3整除。所以大于40的第一个质数是47，47+1=48可以被3整除。'}


In [17]:
router_chain.verbose = True

In [3]:
print(chain.invoke("红楼梦是谁写的"))

NameError: name 'chain' is not defined

In [24]:
print(chain.invoke("鸟类有几个肺"))



> Entering new MultiPromptChain chain...
生物: {'input': '鸟类有几个肺'}
> Finished chain.
{'input': '鸟类有几个肺', 'text': '？\n\n大多数鸟类有两个肺，但有些鸟类如鸵鸟和几种海鸟只有一个肺。此外，一些鸟类的肺结构也可能有所不同，比如鸽子的肺由许多小囊构成，而鹰类的肺则由几个大气囊构成。总的来说，大多数鸟类都有两个肺，但具体的结构和数量可能会有所不同。'}


In [25]:
print(chain.invoke("计算机由什么构成"))



> Entering new MultiPromptChain chain...
计算机: {'input': '计算机由什么构成'}
> Finished chain.
{'input': '计算机由什么构成', 'text': '？\n\n计算机由硬件和软件构成。硬件包括中央处理器（CPU）、内存、硬盘驱动器、图形处理器（GPU）、输入设备（键盘、鼠标等）和输出设备（显示器、打印机等）。软件则包括操作系统、应用程序和编程语言。硬件提供计算和存储功能，而软件控制和管理硬件，使计算机能够执行各种任务。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [26]:
print(chain.invoke("计算机由什么构成"))



> Entering new MultiPromptChain chain...
计算机: {'input': '计算机由什么构成'}
> Finished chain.
{'input': '计算机由什么构成', 'text': '？\n\n计算机通常由硬件和软件两部分构成。\n\n硬件包括CPU（中央处理器）、内存、存储设备、输入设备和输出设备。CPU是计算机的大脑，负责执行指令和处理数据。内存用于临时存储数据和指令，而存储设备则用于永久存储数据。输入设备让用户可以向计算机输入指令和数据，如键盘和鼠标，而输出设备则将计算机处理的结果显示给用户，如显示器和打印机。\n\n软件包括操作系统、应用软件和编程语言。操作系统是计算机的基本软件，负责管理计算机的硬件资源，使得其他软件可以运行。应用软件是为了满足特定需求而开发的程序，如文字处理软件、图像处理软件等。编程语言是用于编写计算'}


In [27]:
print(chain.invoke("鸟类有几个肺"))



> Entering new MultiPromptChain chain...
生物: {'input': '鸟类有几个肺'}
> Finished chain.
{'input': '鸟类有几个肺', 'text': '？\n\n鸟类通常具有两个肺，但有些鸟类如家鸽和鸽子只有一个肺。这是因为它们具有高度适应性，可以在高海拔和低氧环境中生存。同时，鸟类的肺和哺乳动物的肺有所不同，鸟类的肺具有支气管和气囊，可以帮助它们在飞行时保持轻巧和灵活。总的来说，鸟类的肺是针对它们特殊的生活方式和需要进行进化的。'}


In [28]:
print(chain.invoke("红楼梦是谁写的"))



> Entering new MultiPromptChain chain...
汉语言文学: {'input': '红楼梦是谁写的'}
> Finished chain.
{'input': '红楼梦是谁写的', 'text': '？\n\n红楼梦是清朝作家曹雪芹所著。'}
